In [1]:
few_shot_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.train.short.csv'
verify_few_shot_path = '/scratch/dzhang5/LLM/TWEET-FID/Food-verify.expert.train.short.csv'
data_path = '/scratch/dzhang5/LLM/TWEET-FID/traintestfortest-results-autolabel-expert/0.1/1.0/gpt-3.5-turbo_semantic_similarity_COT__cur_False_ground_False_final_COT_label_diversity_similarity_expert.traintestfortest.csv'
ori_data_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.traintestfortest.csv'
output_dir = '/scratch/dzhang5/LLM/TWEET-FID/traintestfortest-results-autolabel-ner-qa-expert/0.1/1.0/lv_v1_tv_v1'
model_name = "gpt-3.5-turbo"
label_column = 'Food_answer'
text_column = 'context'
explanation_column = 'Food_explanation'
reference_column = 'word_check_reference'
example_selection_label_column = 'Food_check_class'
label_symbol = "^^^^"
few_shot_num = 8
few_shot_selection = 'semantic_similarity'
verify_few_shot_selection = 'label_diversity_similarity'
token_path = "/home/dzhang5/.cache/huggingface/token"
cache=False
console_output=True
temperature=0.1
verify=True
label_version='v1'
task_version='v1'
random_shuffle_examples = True
random_shuffle_examples_seed = 1

# Generation Stage

In [2]:
from autolabel.schema import ModelProvider, TaskType
from autolabel.models import register_model, MODEL_REGISTRY
from hf_pipeline_new import HFPipelineLLMNew
from few_shot_new import NewAutoLabelConfig, NewExampleSelectorFactory
from autolabel.few_shot import ExampleSelectorFactory
from template_inst import update_inst_mode
from named_entity_recognition_new import NewNamedEntityRecognitionTask
from classification_new import NewClassificationTask
from question_answering_new import NewQuestionAnsweringTask
from autolabel.tasks import TASK_TYPE_TO_IMPLEMENTATION
from prompt_template import load_ner_check_task_prompt
import sys

In [3]:
update_inst_mode(model_name)
TASK_TYPE_TO_IMPLEMENTATION[TaskType.NAMED_ENTITY_RECOGNITION] = NewNamedEntityRecognitionTask
TASK_TYPE_TO_IMPLEMENTATION[TaskType.CLASSIFICATION] = NewClassificationTask
TASK_TYPE_TO_IMPLEMENTATION[TaskType.QUESTION_ANSWERING] = NewQuestionAnsweringTask
sys.modules['autolabel.labeler'].ExampleSelectorFactory = NewExampleSelectorFactory
register_model(ModelProvider.HUGGINGFACE_PIPELINE, HFPipelineLLMNew)

In [4]:
from autolabel import LabelingAgent, AutolabelDataset
import json
import os
import pandas as pd

In [5]:
with open(token_path) as tfile:
    token_str = tfile.read()

from huggingface_hub import login
login(token=token_str)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dzhang5/.cache/huggingface/token
Login successful


In [6]:
no_auto = ["microsoft/prophetnet", "microsoft/phi-2", "google/pegasus-x"]
if any([pre in model_name for pre in no_auto]):
    device_map = None
else:
    device_map = "auto"

In [7]:
if not os.path.exists(output_dir):
    # Create the directory
    os.makedirs(output_dir)
label_type = label_column.split('_')[0]
output_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_COT_check_' + '_' + label_type + '_' + os.path.split(data_path)[-1]
output_path = os.path.join(output_dir, output_name)

In [8]:
refuel_models = ["refuel-llm", "llama-13b-chat"]
if model_name in refuel_models:
    provider = "refuel"
    em_provider = "huggingface_pipeline"
    model_params = {"max_length":4096, "temperature": temperature}
    task_name = f"FoodborneIllnessIncidentTweetNERQA_{few_shot_selection}_{label_type}_{model_name}"
elif model_name.startswith('gpt'):
    provider = "openai"
    em_provider = "openai"
    model_params = {"max_tokens":4096, "temperature": temperature}
    task_name = f"FoodborneIllnessIncidentTweetNERQA_{few_shot_selection}_{label_type}_{model_name}"
else:
    provider = "huggingface_pipeline"
    em_provider = "huggingface_pipeline"
    model_params = {"max_length":4096, "temperature": temperature,
                    "quantize": 16, "device_map": device_map,
                    "token": token_str}
    task_name = f"FoodborneIllnessIncidentTweetNERQA_{few_shot_selection}_{label_type}_{model_name.split('/')[1]}"

In [9]:
sym_len = len(label_symbol)
label_prefix, label_suffix, label_description, task_guideline, output_guideline = load_ner_check_task_prompt(label_type, label_symbol, label_version, task_version, True)

In [10]:
from utils import generate_word_check_ref_ans, generate_word_explanation_prediction
import random

In [11]:
test_data = pd.read_csv(data_path)
ori_test_data = pd.read_csv(ori_data_path)

assert (test_data[text_column] == ori_test_data[text_column]).all()

if label_column not in test_data.columns:
    test_data[label_column] = ori_test_data[label_column]

if reference_column not in test_data.columns:
    test_data[reference_column] = test_data['sentence_class_label'].apply(generate_word_explanation_prediction)

# Verification Stage

In [12]:
last_results = pd.read_csv(output_path) 

In [13]:
from utils import get_predictions, check_prediction, get_verify_df, construct_results

In [14]:
verify_question_column = f'{label_type}_verify_question'
verify_answer_column = f'{label_type}_verify_answer'
verify_explanation_column = f'{label_type}_verify_explanation'
pos_column = f'{label_column}_pos'
verify_example_selection_label_column = f'has_{label_type}'

In [15]:
test_data[[verify_question_column, pos_column]] = last_results[[f'{label_column}_label']].apply(lambda x: get_predictions(x[f'{label_column}_label'], label_symbol), axis=1,result_type="expand")

In [16]:
if 'CategorizedLabels' not in test_data.columns:
    test_data['CategorizedLabels'] = ori_test_data['CategorizedLabels']
test_data[verify_answer_column] = test_data[['CategorizedLabels', verify_question_column]].apply(lambda x: check_prediction(x['CategorizedLabels'], x[verify_question_column], label_type), axis=1)

In [17]:
new_test_data = get_verify_df(test_data, verify_question_column, verify_answer_column, text_column, pos_column)

In [18]:
verify_task_guideline = (f"You are an expert at identifying {label_type} entities that are related to foodborne illness incident from text. " 
                         f"In the given text, your task is to verify if a given word is a {label_type} entity that {label_description[label_type].replace('are ', 'is ').replace('.', '')} in the given text. "
                         "Use the following examples as a guide for your analysis and format your responses similarly.")
question = f'''Do you think the word "{{{verify_question_column}}}" in the given text is a {label_type} entity that {label_description[label_type].replace('are ', 'is ').replace('.', '')}?'''

In [19]:
if verify_few_shot_selection == few_shot_selection:
    output_verify_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_COT_check_' + '_' + label_type + '_verify_COT_' + os.path.split(data_path)[-1]
    output_verify_path = os.path.join(output_dir, output_verify_name)
    output_final_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_COT_check_' + '_' + label_type + '_final_COT_' + os.path.split(data_path)[-1]
else:
    output_verify_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_COT_check_' + '_' + label_type + '_verify_COT_' + verify_few_shot_selection + '_' + os.path.split(data_path)[-1]
    output_verify_path = os.path.join(output_dir, output_verify_name)
    output_final_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_COT_check_' + '_' + label_type + '_final_COT_' + verify_few_shot_selection + '_' + os.path.split(data_path)[-1]
output_final_path = os.path.join(output_dir, output_final_name)

In [20]:
config = {
    "task_name": task_name+'_verification',
    "task_type": "classification",
    "dataset": {
        "label_column": verify_answer_column,
        "text_column": text_column,
        "explanation_column": verify_explanation_column,
        "example_selection_label_column": verify_example_selection_label_column,
        "delimiter": ",",
        "label_description": label_description
    },
    "model": {
        "provider": provider,
        "name": model_name,
        "params": model_params
    },
    "embedding": {
        "provider": em_provider,
    },
    "prompt": {
        "task_guidelines": verify_task_guideline,
        "output_guidelines": ('''Your answer will consist of an explanation, followed by the correct answer ("Yes" or "No").'''
                              f'''Please answer with "Yes" if the given word is a {label_type} entity that {label_description[label_type].replace('are ', 'is ').replace('.', '')} in the given text, otherwise answer with "No".'''
                              '''The last line of the response should always be JSON format with one key: {"label": "the correct answer"}.\n'''
                             ),
        "labels": [
            "Yes",
            "No"
        ],
        "example_selection_labels":[
            "yes",
            "no"
        ],
        "few_shot_examples": verify_few_shot_path,
        "few_shot_selection": verify_few_shot_selection,
        "few_shot_num": few_shot_num,
        "random_shuffle_examples": random_shuffle_examples,
        "random_shuffle_examples_seed": random_shuffle_examples_seed,
        "example_template": f"Context: {{{text_column}}}\nQuestion: {question}\nAnswer: Let's think step by step.\n{{{verify_explanation_column}}}\n{{{verify_answer_column}}}",
        "chain_of_thought": True
    }
}

config = NewAutoLabelConfig(config)

In [21]:
agent = LabelingAgent(config=config, console_output=console_output, cache=cache)

2024-03-31 19:39:30 autolabel.labeler WARNING: cache parameter is deprecated and will be removed soon. Please use generation_cache and transform_cache instead.
/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [22]:
verify_ds = AutolabelDataset(new_test_data.drop(['text_idx',pos_column], axis=1), config=config)

In [23]:
agent.plan(verify_ds)

/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
2024-03-31 19:39:33 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-31 19:39:38 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-31 19:39:42 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Output()

2024-03-31 19:39:44 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-31 19:39:45 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-31 19:39:45 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-31 19:39:45 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-31 19:39:45 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-31 19:39:45 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-31 19:39:46 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-31 19:39:46 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-31 19:39:46 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-03-31 19:39:46 httpx INFO: HTTP Request: 

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $9.9449 │
│ Number of Examples       │ 1018    │
│ Average cost per example │ $0.0098 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at identifying Food entities that are related to foodborne illness incident from text. In the 
given text, your task is to verify if a given word is a Food entity that is specific food item that caused the 
potential foodborne illness incident in the given text. Use the following examples as a guide for your analysis and
format your responses similarly.

Your answer will consist of an explanation, followed by the correct answer ("Yes" or "No").Please answer with "Yes"
if the given word is a Food entity that is specific food item that caused the potential foodborne illness incident 
in the given text, otherwise answer with "No".The last line of the response should always be JSON format with one 
key: {"label": "the correct answer"}.


Some examples with their output answers are provided below:

Context: Food poisoning from this weekend still fucking w/ me smh [ EMOJI_weary_face ]
Question: Do you think the word "poisoning from" in the given text is a Food entity that is specific food item that
caused the potential foodborne illness incident?
Answer: Let's think step by step.
The word "poisoning from" does not cause a potential foodborne illness incident.
{"label": "No"}
Context: @USER @USER @USER LOL DONT GET FOOD POISONING ! Me and my friend both got a taco in a bag on Tuesday and 
we were both extremely sick the next morning
Question: Do you think the word "taco" in the given text is a Food entity that is specific food item that caused 
the potential foodborne illness incident?
Answer: Let's think step by step.
The word "taco" is a specific food item that caused the potential foodborne illness incident.
{"label": "Yes"}
Context: I wish this Food Poison shit would go away !
Question: Do you think the word "!" in the given text is a Food entity that is specific food item that caused the 
potential foodborne illness incident?
Answer: Let's think step by step.
The word "!" does not cause a potential foodborne illness incident.
{"label": "No"}
Context: Food poison , hate that school lunch
Question: Do you think the word "school lunch" in the given text is a Food entity that is specific food item that 
caused the potential foodborne illness incident?
Answer: Let's think step by step.
The word "school lunch" is a specific food item that caused the potential foodborne illness incident.
{"label": "Yes"}
Context: one time ruel made me a quesadilla and it gave me food poisoning . #ruelisoverparty
Question: Do you think the word "quesadilla" in the given text is a Food entity that is specific food item that 
caused the potential foodborne illness incident?
Answer: Let's think step by step.
The word "quesadilla" is a specific food item that caused the potential foodborne illness incident.
{"label": "Yes"}
Context: S/O to Juannita's for some DOPE Little 5 food poisoning ! ! ! ! Talk about the ILLEST college weekend , 
amirite ? !
Question: Do you think the word "? !" in the given text is a Food entity that is specific food item that caused the
potential foodborne illness incident?
Answer: Let's think step by step.
The word "? !" does not cause a potential foodborne illness incident.
{"label": "No"}
Context: Got food poisoning from school food . [ EMOJI_face_with_medical_mask ]
Question: Do you think the word "school food" in the given text is a Food entity that is specific food item that 
caused the potential foodborne illness incident?
Answer: Let's think step by step.
The word "school food" is a specific food item that caused the potential foodborne illness incident.
{"label": "Yes"}
Context: Boyyyy oh boy . This like that time I got food poisoning at the most inopportune of possible times .
Question: Do you think the word "at the" in the given text is a Food entity that is specific food item that caused 
the potential foodborne illness incident?
Answer: Let's think step by step.
The word "at the" does not cause a potential foodborne illness incident.
{"label": "No"}

Now I want you to label the following example:
Conte

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [ ]:
# now, do the actual labeling
verify_ds = agent.run(verify_ds)

In [ ]:
metrics = verify_ds.eval()

In [ ]:
new_test_data[f'{verify_answer_column}_label'] = verify_ds.df[f'{verify_answer_column}_label']

In [ ]:
verify_ds.df['text_idx'] = new_test_data['text_idx']

In [ ]:
verify_ds.df.to_csv(output_verify_path, index=False)
verify_ds.df.to_pickle(output_verify_path.replace('.csv', '.pkl'))

In [ ]:
new_result = construct_results(last_results, new_test_data, f'{label_column}_label', f'{verify_answer_column}_label', "text_idx", pos_column, label_symbol)

In [ ]:
new_result.to_csv(output_final_path, index=False)
new_result.to_pickle(output_final_path.replace('.csv', '.pkl'))